In [310]:
import os
import datetime
import math
from get_today_schedule import get_today_sched
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import date, datetime


'''
This function will run the simulations on a test data set to see how they
perform on last seasons data.
'''

#pull in the entire schedule dataset
engine = create_engine(os.environ.get('DEV_DB_CONNECT'))

sql_query = 'SELECT * from nhl_tables.nhl_schedule'
df = pd.read_sql(sql_query, con=engine,
                 parse_dates = {'game_date': '%Y-%m-%d'})



In [311]:
season_start = '2017-10-04'
season_end = '2018-04-08'
startdate = datetime.strptime(season_start, '%Y-%m-%d')
enddate = datetime.strptime(season_end, '%Y-%m-%d')

In [312]:
date_list = [date.fromordinal(i) for i in range(startdate.toordinal(), enddate.toordinal()+1)]

In [313]:
startdate_str = startdate.strftime('%Y-%m-%d')
    
    

In [314]:
def get_avg_df(df):
    reg = df[df.ot_flag != 1]
    ot = df[(df.ot_flag == 1) & (df.shootout_flag != 1)]
    shootout = df[df.shootout_flag == 1]

    reg_avg = reg[['home_score', 'away_score','seconds_in_ot']].mean()
    ot_avg = ot[['home_score', 'away_score', 'seconds_in_ot']].mean()
    shootout_avg = shootout[['home_score', 'away_score', 'seconds_in_ot']].mean()

    reg_avg['ot_flag'] = 0
    ot_avg['ot_flag'] = 1
    shootout_avg['ot_flag'] = 1

    reg_avg['shootout_flag'] = 0
    ot_avg['shootout_flag'] = 0
    shootout_avg['shootout_flag'] = 1

    avg_df = pd.concat([reg_avg, ot_avg, shootout_avg], axis=1).T

    avg_df.columns = ['goals_for', 'goals_against', 'seconds_in_ot', 'ot_flag', 'shootout_flag']
    avg_df['non_ot_goals_for'] = np.where(((avg_df.shootout_flag == 1) | (avg_df.ot_flag == 1)) & 
                                              (avg_df.goals_for > avg_df.goals_against), avg_df.goals_for - 1,
                                              avg_df.goals_for)
    avg_df['non_ot_goals_against'] = np.where(((avg_df.shootout_flag == 1) | (avg_df.ot_flag == 1)) & 
                                              (avg_df.goals_for < avg_df.goals_against), avg_df.goals_against - 1,
                                              avg_df.goals_against)
    avg_df['ot_goals'] = np.where(avg_df.shootout_flag == 0, 
                                          avg_df.goals_for - avg_df.non_ot_goals_for, 0)
    avg_df['ot_goals_against'] = np.where((avg_df.ot_flag == 1) & (avg_df.shootout_flag != 1), 1, 0)
    
    return avg_df

In [315]:
def clean_results(results_df, team, date):
    '''
    this function cleans the results dataframe and just strips out the wanted
    team results and creates a column for OT goals as well 
    '''
    results_df = results_df[results_df.game_date < date]
    cleaned_results = []
    #looping through the results_df to pull out only the games the team variable played in.
    for index, row in results_df.iterrows():
        if row.home_team == team:
            #print(row)
            new_row = row[['game_id', 'game_type', 'season', 'game_date', 'home_team_id', 'home_team',
                           'home_score', 'away_score', 'ot_flag', 'shootout_flag', 'seconds_in_ot']]
            
            #print(new_row)
            new_row.index = ['game_id', 'game_type', 'season', 'game_date', 'team_id', 'team',
                           'goals_for', 'goals_against', 'ot_flag', 'shootout_flag', 'seconds_in_ot']
            
            new_row['is_home'] = 1
            
            cleaned_results.append(new_row)
            
        elif row.away_team == team:
            #print(row)
            new_row = row[['game_id', 'game_type', 'season', 'game_date', 'away_team_id', 'away_team',
                           'away_score', 'home_score', 'ot_flag', 'shootout_flag', 'seconds_in_ot']]
            #print(new_row)
            new_row.index = ['game_id', 'game_type', 'season', 'game_date', 'team_id', 'team',
                           'goals_for', 'goals_against', 'ot_flag', 'shootout_flag', 'seconds_in_ot']
            
            new_row['is_home'] = 0
            
            #print(new_row)
            cleaned_results.append(new_row)
            
    cleaned_df = pd.concat(cleaned_results, axis=1).T
    
    #calculating non ot goals by seeing if the game went to ot or shootout and if so whether the team won or not.
    #if they did then they score one less goals than their final total if not then they scored their same goals for 
    #amount
    cleaned_df['non_ot_goals_for'] = np.where(((cleaned_df.shootout_flag == 1) | (cleaned_df.ot_flag == 1)) & 
                                          (cleaned_df.goals_for > cleaned_df.goals_against), cleaned_df.goals_for - 1,
                                          cleaned_df.goals_for)
    cleaned_df['non_ot_goals_against'] = np.where(((cleaned_df.shootout_flag == 1) | (cleaned_df.ot_flag == 1)) & 
                                          (cleaned_df.goals_for < cleaned_df.goals_against), cleaned_df.goals_against - 1,
                                          cleaned_df.goals_against)
    cleaned_df['ot_goals'] = np.where(cleaned_df.shootout_flag == 0, 
                                      cleaned_df.goals_for - cleaned_df.non_ot_goals_for, 0)
    cleaned_df['ot_goals_against'] = np.where(cleaned_df.shootout_flag == 0,
                                              cleaned_df.goals_against - cleaned_df.non_ot_goals_against,0)
                                            
    cleaned_df = cleaned_df.reset_index(drop=True)
    
    #only return the last two seasons of games
    cleaned_df = cleaned_df.sort_values(by=['game_date'], ascending=False).iloc[:164, :]
    
    return cleaned_df
            

In [316]:
def monte_carlo_predict(home_results, away_results):
    '''
    taking the results of each team this function runs a monte carlo
    simulation to predict goals scored using the poissson distribution
    '''
    results =[]
    
    home_lambda = (home_results['non_ot_goals_for'].mean() + away_results['non_ot_goals_against'].mean()) / 2
    away_lambda = (home_results['non_ot_goals_against'].mean() + away_results['non_ot_goals_for'].mean()) / 2
    
    home_ot_lambda = (((home_results['ot_goals'].sum()/home_results['seconds_in_ot'].sum()) * 300) + \
                      (away_results['ot_goals_against'].sum()/away_results['seconds_in_ot'].sum())*300)/2
    
    away_ot_lambda = (((away_results['ot_goals'].sum()/away_results['seconds_in_ot'].sum()) * 300) + \
                      (home_results['ot_goals_against'].sum()/home_results['seconds_in_ot'].sum())*300)/2
    
    home_ot_win_percent = np.where((home_results.ot_flag == 1) & (home_results.shootout_flag == 0)
                                   & (home_results.goals_for > home_results.goals_against), 1, 0).sum()/home_results[(home_results.ot_flag == 1) & (home_results.shootout_flag == 0)].shape[0]
    away_ot_win_percent = np.where((away_results.ot_flag == 1) & (away_results.shootout_flag == 0)
                                   & (away_results.goals_for > away_results.goals_against), 1, 0).sum()/away_results[(away_results.ot_flag == 1) & (away_results.shootout_flag == 0)].shape[0]
    home_so_win_percent = np.where((home_results.shootout_flag == 1) &(home_results.goals_for > home_results.goals_against), 
                                   1, 0).sum()/home_results[home_results.shootout_flag == 1].shape[0]
    away_so_win_percent = np.where((away_results.shootout_flag == 1) & (away_results.goals_for > away_results.goals_against), 
                                   1, 0).sum()/away_results[away_results.shootout_flag == 1].shape[0]
    if math.isnan(home_ot_win_percent):
        home_ot_win_percent = .5
        
    if math.isnan(away_ot_win_percent):
        away_ot_win_percent = .5
        
    if math.isnan(home_so_win_percent):
        home_so_win_percent = .5
        
    if math.isnan(away_so_win_percent):
        away_so_win_percent = .5
    
    home_reg_goals = np.random.poisson(home_lambda, 1000)
    away_reg_goals = np.random.poisson(away_lambda, 1000)
       
    for home, away in zip(home_reg_goals, away_reg_goals):
        if home > away:
            results.append(1)
        elif away > home:
            results.append(0)
        else:
            prob_of_zero_goals = (math.exp(-home_ot_lambda) * math.exp(-away_ot_lambda))
            
            if np.random.binomial(1, prob_of_zero_goals) == 1:
                try:
                    prob_of_home_so_win = home_so_win_percent/(home_so_win_percent + away_so_win_percent)
                except:
                    prob_of_home_so_win = .5
                results.append(np.random.binomial(1, prob_of_home_so_win))
            else:
                try:
                    prob_of_home_ot_win = home_ot_win_percent/(home_ot_win_percent + away_ot_win_percent)
                except:
                    prob_of_home_ot_win = .5
                results.append(np.random.binomial(1, prob_of_home_ot_win))
                
    home_win_prob = sum(results)/len(results)
                
    return home_win_prob
    
    
    

In [ ]:
#looping over every date of the 2018 season to pull the games from the dataframe 
#that were played on that date
predictions = []

for game_date in date_list:
    #create a dataframe of the games played
    schedule_df = df[df.game_date == pd.Timestamp(game_date).strftime('%Y-%m-%d')]
    train_df = df[df.game_date < pd.Timestamp(game_date).strftime('%Y-%m-%d')]
    ot_df = train_df[(train_df.ot_flag == 1) & (train_df.shootout_flag != 1)]
    avg_scoring_per_game = ((train_df['home_score'].sum() + train_df['away_score'].sum())/df.shape[0])/60
    avg_scoring_per_ot = (ot_df.shape[0]/ot_df.seconds_in_ot.sum()) * 60
    home_win_pct = train_df.home_win.sum()/train_df.shape[0]
    
    #home ice advantage multiplier
    home_mult = 1+((home_win_pct - .5)/.5)
    
    #iterating over the dataframe to simulate one game at a time
    for index, row in schedule_df.iterrows():
        game_id = row.game_id
        
        home_team = row.home_team
        away_team = row.away_team
        #all these try and excepts are because vegas has no past samples
        try:
            home_results = clean_results(df, home_team, game_date)
            
            if home_results.shape[0] < 2:
                home_results = get_avg_df(train_df)
                
        except:
            home_results = get_avg_df(train_df)
        try:
            away_results = clean_results(df, away_team, game_date)
            #vegas bullshit
            if away_results.shape[0] < 2:
                away_results = get_avg_df(train_df)
                
        except:
            away_results = get_avg_df(train_df)
        home_win_probabilities = []
        
        #running the monte carlo simulation a 1,000 times
        print(f'{away_team} vs. {home_team}')
        print(pd.Timestamp(game_date).strftime('%Y-%m-%d'))
        print(row.game_id)
        for x in range(0,1000):
            win_prob = monte_carlo_predict(home_results, away_results)
            win_prob = home_mult * win_prob
            home_win_probabilities.append(win_prob)
        #multiply result from monte carlo function by a home team multiplier
        print(sum(home_win_probabilities)/len(home_win_probabilities))
        final_win_probs = sum(home_win_probabilities)/len(home_win_probabilities)
        predictions.append([game_id, game_date, home_team, away_team, final_win_probs])
        

/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


Toronto Maple Leafs vs. Winnipeg Jets
2017-10-04
2017020001
0.5527402886178868
St. Louis Blues vs. Pittsburgh Penguins
2017-10-05
2017020002
0.5693922031694482
Calgary Flames vs. Edmonton Oilers
2017-10-05
2017020003
0.5502348728159318
Philadelphia Flyers vs. San Jose Sharks
2017-10-05
2017020004
0.5874450353514868
Nashville Predators vs. Boston Bruins
2017-10-05
2017020005
0.5597368216172329
Colorado Avalanche vs. New York Rangers
2017-10-05
2017020007
0.640605674928886
Minnesota Wild vs. Detroit Red Wings
2017-10-05
2017020009
0.4907171117431933
Montréal Canadiens vs. Buffalo Sabres
2017-10-05
2017020006
0.4917954937017468
Washington Capitals vs. Ottawa Senators
2017-10-05
2017020008
0.4580283185696864
Pittsburgh Penguins vs. Chicago Blackhawks
2017-10-06
2017020010
0.526381427298501
Arizona Coyotes vs. Anaheim Ducks
2017-10-06
2017020011
0.6187201587687325
Philadelphia Flyers vs. Los Angeles Kings
2017-10-06
2017020012
0.5660261401377072
New York Islanders vs. Columbus Blue Jackets


/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars


0.6927593394642141
New Jersey Devils vs. Toronto Maple Leafs
2017-10-11
2017020046
0.5713043742503001
Pittsburgh Penguins vs. Washington Capitals
2017-10-11
2017020047
0.5764980503798478
Ottawa Senators vs. Vancouver Canucks
2017-10-11
2017020044
0.48831112754897993
Boston Bruins vs. Colorado Avalanche
2017-10-12
2017020048
0.46086007179896293
New York Islanders vs. Anaheim Ducks
2017-10-12
2017020049
0.5578166956521743
Calgary Flames vs. Los Angeles Kings
2017-10-12
2017020050
0.5702216418029514
Pittsburgh Penguins vs. Tampa Bay Lightning
2017-10-12
2017020051
0.5153841627443152
St. Louis Blues vs. Florida Panthers
2017-10-12
2017020052
0.5169369573195054
Dallas Stars vs. Nashville Predators
2017-10-13
2017020053
0.5459507197452214
Minnesota Wild vs. Chicago Blackhawks
2017-10-13
2017020054
0.5681327961783439
Winnipeg Jets vs. Vancouver Canucks
2017-10-13
2017020055
0.502468796178343
Detroit Red Wings vs. Arizona Coyotes
2017-10-13
2017020056
0.5087119299363052
Buffalo Sabres vs. San 

Dallas Stars vs. Edmonton Oilers
2017-10-27
2017020147
0.5502725873320535
Washington Capitals vs. Vancouver Canucks
2017-10-27
2017020148
0.4024064452975045
Colorado Avalanche vs. Vegas Golden Knights
2017-10-27
2017020149
0.6813781765834935
Winnipeg Jets vs. Columbus Blue Jackets
2017-10-27
2017020151
0.6009279577735126
St. Louis Blues vs. Carolina Hurricanes
2017-10-27
2017020152
0.4857322917466424
Ottawa Senators vs. New Jersey Devils
2017-10-27
2017020150
0.501366276391555
Nashville Predators vs. Chicago Blackhawks
2017-10-28
2017020153
0.5927321768759585
Dallas Stars vs. Calgary Flames
2017-10-28
2017020154
0.5608728346094926
San Jose Sharks vs. Buffalo Sabres
2017-10-28
2017020155
0.4659182633996945
Los Angeles Kings vs. Boston Bruins
2017-10-28
2017020156
0.5320437258805505
Philadelphia Flyers vs. Toronto Maple Leafs
2017-10-28
2017020157
0.5337454441041348
New York Rangers vs. Montréal Canadiens
2017-10-28
2017020158
0.5125218407350696
Anaheim Ducks vs. Tampa Bay Lightning
2017

/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars


0.4110031499238962
Arizona Coyotes vs. Philadelphia Flyers
2017-10-30
2017020171
0.6116932153729084
Montréal Canadiens vs. Ottawa Senators
2017-10-30
2017020173
0.5505374687975636
Tampa Bay Lightning vs. Florida Panthers
2017-10-30
2017020174
0.5103783873668194
Boston Bruins vs. Columbus Blue Jackets
2017-10-30
2017020172
0.5619208888888882
Los Angeles Kings vs. St. Louis Blues
2017-10-31
2017020175
0.5474424373576313
Dallas Stars vs. Vancouver Canucks
2017-10-31
2017020176
0.4926811465451786
Toronto Maple Leafs vs. San Jose Sharks
2017-10-31
2017020177
0.5896887851176917
Vegas Golden Knights vs. New York Rangers
2017-10-31
2017020178
0.4612515489749436
Arizona Coyotes vs. Detroit Red Wings
2017-10-31
2017020179
0.5814540698557322
Winnipeg Jets vs. Minnesota Wild
2017-11-01
2017020180
0.591105776430462
Philadelphia Flyers vs. Chicago Blackhawks
2017-11-02
2017020181
0.5781171000000007
Pittsburgh Penguins vs. Edmonton Oilers
2017-11-02
2017020182
0.5001813727272733
New Jersey Devils vs.

In [300]:
predict_df.columns = ['game_id', 'game_date', 'home_team', 'away_team', 'home_win_probs']
predict_df.sort_values(by='game_id').tail()

,game_id,game_date,home_team,away_team,home_win_probs
1258,2017021260,2018-04-07,Detroit Red Wings,New York Islanders,0.525466
1259,2017021261,2018-04-07,Florida Panthers,Buffalo Sabres,0.609958
1260,2017021262,2018-04-07,Washington Capitals,New Jersey Devils,0.637545
1261,2017021263,2018-04-07,Carolina Hurricanes,Tampa Bay Lightning,0.468229
1262,2017021264,2018-04-07,Winnipeg Jets,Chicago Blackhawks,0.569880


In [299]:
df.sort_values(by="game_id").tail()

,game_id,game_type,season,game_date,home_team_id,home_team,home_score,away_team_id,away_team,away_score,ot_flag,shootout_flag,seconds_in_ot,home_win
3418,2017021267,R,20172018,2018-04-08,53,Arizona Coyotes,0,24,Anaheim Ducks,3,0,0,0,0
3419,2017021268,R,20172018,2018-04-08,20,Calgary Flames,7,54,Vegas Golden Knights,1,0,0,0,1
3730,2017021269,R,20172018,2018-04-08,22,Edmonton Oilers,3,23,Vancouver Canucks,2,0,1,300,1
3420,2017021270,R,20172018,2018-04-08,26,Los Angeles Kings,2,25,Dallas Stars,4,0,0,0,0
3421,2017021271,R,20172018,2018-04-08,28,San Jose Sharks,3,30,Minnesota Wild,6,0,0,0,0


In [279]:
print([home_ot_win_percent, away_ot_win_percent, home_so_win_percent, away_so_win_percent])

[0.5, 1.0, 0.5, 0.5]


In [281]:
home_results.tail()


,game_id,game_type,season,game_date,team_id,team,goals_for,goals_against,ot_flag,shootout_flag,seconds_in_ot,is_home,non_ot_goals_for,non_ot_goals_against,ot_goals,ot_goals_against
164,2015020246,R,20152016,2015-11-14 00:00:00,3,New York Rangers,2,1,0,1,300,0,1,1,0,0
14,2015020231,R,20152016,2015-11-13 00:00:00,3,New York Rangers,6,3,0,0,0,1,6,3,0,0
13,2015020217,R,20152016,2015-11-11 00:00:00,3,New York Rangers,3,0,0,0,0,1,3,0,0,0
12,2015020208,R,20152016,2015-11-08 00:00:00,3,New York Rangers,4,1,0,0,0,0,4,1,0,0
11,2015020197,R,20152016,2015-11-07 00:00:00,3,New York Rangers,2,1,0,0,0,0,2,1,0,0


home_results.head()

In [260]:
df[((df.home_team == 'New York Rangers') | (df.away_team == 'New York Rangers'))].sort_values(by='game_date', ascending=False)


,game_id,game_type,season,game_date,home_team_id,home_team,home_score,away_team_id,away_team,away_score,ot_flag,shootout_flag,seconds_in_ot,home_win
3408,2017021257,R,20172018,2018-04-07,4,Philadelphia Flyers,5,3,New York Rangers,0,0,0,0,1
3393,2017021242,R,20172018,2018-04-05,2,New York Islanders,2,3,New York Rangers,1,0,0,0,1
3381,2017021229,R,20172018,2018-04-03,1,New Jersey Devils,5,3,New York Rangers,2,0,0,0,1
3364,2017021212,R,20172018,2018-03-31,12,Carolina Hurricanes,1,3,New York Rangers,2,0,0,0,0
3353,2017021201,R,20172018,2018-03-30,3,New York Rangers,3,14,Tampa Bay Lightning,7,0,0,0,0
3339,2017021187,R,20172018,2018-03-29,15,Washington Capitals,3,3,New York Rangers,2,1,0,38,1
3325,2017021170,R,20172018,2018-03-26,3,New York Rangers,2,15,Washington Capitals,4,0,0,0,0
3315,2017021159,R,20172018,2018-03-24,3,New York Rangers,5,7,Buffalo Sabres,1,0,0,0,1
3293,2017021137,R,20172018,2018-03-22,4,Philadelphia Flyers,4,3,New York Rangers,3,0,0,0,1
3279,2017021122,R,20172018,2018-03-20,3,New York Rangers,3,29,Columbus Blue Jackets,5,0,0,0,0


In [248]:
home_results.head(10)

,game_id,game_type,season,game_date,team_id,team,goals_for,goals_against,ot_flag,shootout_flag,seconds_in_ot,is_home,non_ot_goals_for,non_ot_goals_against,ot_goals,ot_goals_against
0,2015020002,R,20152016,2015-10-08 00:00:00,3,New York Rangers,3,2,0,0,0,0,3,2,0,0
1,2015020013,R,20152016,2015-10-09 00:00:00,3,New York Rangers,4,2,0,0,0,0,4,2,0,0
2,2015020021,R,20152016,2015-10-10 00:00:00,3,New York Rangers,5,2,0,0,0,1,5,2,0,0
3,2015020038,R,20152016,2015-10-13 00:00:00,3,New York Rangers,1,4,0,0,0,1,1,4,0,0
4,2015020053,R,20152016,2015-10-15 00:00:00,3,New York Rangers,0,3,0,0,0,0,0,3,0,0
5,2015020075,R,20152016,2015-10-18 00:00:00,3,New York Rangers,1,2,1,0,187,1,1,1,0,1
6,2015020080,R,20152016,2015-10-19 00:00:00,3,New York Rangers,4,0,0,0,0,1,4,0,0,0
7,2015020092,R,20152016,2015-10-22 00:00:00,3,New York Rangers,4,1,0,0,0,1,4,1,0,0
8,2015020119,R,20152016,2015-10-25 00:00:00,3,New York Rangers,4,1,0,0,0,1,4,1,0,0
9,2015020146,R,20152016,2015-10-30 00:00:00,3,New York Rangers,3,1,0,0,0,1,3,1,0,0
